Here is the base code for takings paragraphes of dpef and splitting them into sentences, then keep only long sentences.

MIN_NB_OF_TOKENS=8 seems to do the trick. 

Improvements:
- Remove last filter and look at what has <8 words. Mostly fragments, mainly titles, etc. but may shows errors in parsing.
- Questios were kept here, maybe to exclude if appear in top sentences

In [11]:
import pandas as pd
# text
from spacy.lang.fr import French
from spacy.cli.download import download
import spacy
# download('fr_core_news_sm')

In [110]:
input_filename = "../../data/processed/DPEFs/dpef_paragraphs.csv"
output_filename = "../../data/processed/DPEFs/dpef_paragraphs_sentences.csv"
df = pd.read_csv(input_filename, sep=";")
df = df[df.paragraph.notna()] # nan created while saving/ removing header -> to correct

In [114]:
# params
MIN_NB_OF_TOKENS = 8

# custom parser to split sentences while ignoring title section like cf.
nlp = French()

def prevent_sentence_boundaries(doc):
    for i, token in enumerate(doc[2:]):
        if exception_to_split(token):
            token.is_sent_start = False
    return doc

def exception_to_split(token):
    if 'cf' in token.nbor(-2).text and token.nbor(-1).text == ".":
        return True
    return False
nlp.add_pipe(prevent_sentence_boundaries) # add exception
nlp.add_pipe(nlp.create_pipe('sentencizer')) # to keep default sentencizer aso
# split, keep only the text and forget the nlp tagging for now
df_sent = df["paragraph"].apply(lambda x: [sent.text for sent in nlp(x).sents if len(sent)>MIN_NB_OF_TOKENS])
# save
df["paragraph_sentences"] = df_sent.values
print(df.shape)
df = df[df["paragraph_sentences"].apply(lambda x : len(x)>0)]
print(df.shape)
df.to_csv(output_filename, sep=";")

(13939, 11)
(5981, 11)
